## Insert Title Here ##

## Introduction ##

Insert Introduction Here

## Preliminary Exploratory Data Analysis ##

In [1]:
library(tidyverse)
library(broom)
library(repr)
library(digest)
library(infer)

-- Attaching packages ------------------------------------------------------------------------------- tidyverse 1.3.1 --

v ggplot2 3.3.6      v purrr   0.3.4 
v tibble  3.1.8      v dplyr   1.0.10
v tidyr   1.2.0      v stringr 1.4.1 
v readr   2.1.2      v forcats 0.5.2 

Warning message:
"package 'ggplot2' was built under R version 4.1.3"
Warning message:
"package 'tibble' was built under R version 4.1.3"
Warning message:
"package 'tidyr' was built under R version 4.1.2"
Warning message:
"package 'readr' was built under R version 4.1.2"
Warning message:
"package 'dplyr' was built under R version 4.1.3"
Warning message:
"package 'stringr' was built under R version 4.1.3"
Warning message:
"package 'forcats' was built under R version 4.1.3"
-- Conflicts ---------------------------------------------------------------------------------- tidyverse_conflicts() --
x dplyr::filter() masks stats::filter()
x dplyr::lag()    masks stats::lag()

Warning message:
"package 'broom' was built under 

In [25]:
options(repr.plot.width=15, repr.plot.height=7) # Setting options

The following code imports the April 2023 Canadian Labour Force data into the notebook.

In [26]:
employment <- read_csv("https://raw.githubusercontent.com/GWilliam02/STAT201-Project/main/data/employmentData.csv")

Rows: 108064 Columns: 60
-- Column specification ------------------------------------------------------------------------------------------------
Delimiter: ","
chr  (7): SURVMNTH, AGE_12, NAICS_21, NOC_10, NOC_43, WHYLEFTN, EFAMTYPE
dbl (53): REC_NUM, SURVYEAR, LFSSTAT, PROV, CMA, AGE_6, SEX, MARSTAT, EDUC, ...

i Use `spec()` to retrieve the full column specification for this data.
i Specify the column types or set `show_col_types = FALSE` to quiet this message.


In [34]:
employment_clean <- employment |>
                    select(NOC_10 ,IMMIG ,HRLYEARN) |>
                    filter(!is.na(HRLYEARN)) |>
                    filter(NOC_10 == "03") |>    #Filters out everyone with STEM Occupations
                    select(-NOC_10) |>
                    rename(immigration_status = IMMIG,
                          hourly_wage = HRLYEARN) |>
                    mutate(immigration_status = case_when(immigration_status == 1 | immigration_status == 2 ~ "Resident",
                                                         immigration_status == 3 ~ "Non-resident"))
                    
                

head(employment_clean)

immigration_status,hourly_wage
<chr>,<dbl>
Resident,9707
Non-resident,4567
Non-resident,3897
Resident,2497
Non-resident,4872
Non-resident,2885


In [33]:
nrow(employment)
nrow(employment_clean)

summary_stats <- employment_clean |>
                group_by(immigration_status) |>
                summarize(count = n(), 
                         mean_wage = mean(hourly_wage))

summary_stats


[1] 108064

[1] 4334

immigration_status,count,mean_wage
<chr>,<int>,<dbl>
Non-resident,2937,4248.751
Resident,1397,4558.674
